In [4]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger
from typing import Optional, Tuple

class CryptoDataset(Dataset):
    def __init__(self, features: torch.Tensor, labels: torch.Tensor):
        """Initialize dataset with pre-processed tensors for better efficiency."""
        self.features = features
        self.labels = labels

    def __len__(self) -> int:
        return len(self.features)

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor]:
        return self.features[idx], self.labels[idx]

class CryptoDataModule(pl.LightningDataModule):
    def __init__(
        self, 
        directory_path: str, 
        batch_size: int = 128, 
        num_workers: int = 4, 
        train_split: float = 0.8, 
        n_features: int = 20,
        random_seed: int = 42
    ):
        super().__init__()
        self.directory_path = Path(directory_path)
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.train_split = train_split
        self.n_features = n_features
        self.random_seed = random_seed
        self.scaler = StandardScaler()
        self.selected_features = None
        self.input_dim = None
        self.output_dim = None
        
    def prepare_data(self) -> None:
        """Verify data files exist."""
        features_path = self.directory_path / 'ETH/USDT:USDT_raw_features_20250206_184554.parquet'
        labels_path = self.directory_path / 'ETH/USDT:USDT_raw_labels_20250206_184554.parquet'
        
        if not (features_path.exists() and labels_path.exists()):
            raise FileNotFoundError(f"Data files not found in {self.directory_path}/ETH/")
    
    def setup(self, stage: Optional[str] = None) -> None:
        """Setup data with efficient preprocessing."""
        # Load data
        features = pd.read_parquet(self.directory_path / 'ETH/USDT:USDT_raw_features_20250206_184554.parquet')
        labels = pd.read_parquet(self.directory_path / 'ETH/USDT:USDT_raw_labels_20250206_184554.parquet')
        
        # Print initial info
        print(f"Original features shape: {features.shape}")
        
        # Select features (use numpy for efficiency)
        np.random.seed(self.random_seed)
        self.selected_features = np.random.choice(features.columns, size=self.n_features, replace=False)
        features = features[self.selected_features]
        
        print(f"\nRandomly selected {self.n_features} features:")
        print(self.selected_features.tolist())
        
        # Efficient preprocessing
        features = features.fillna(0).values  # Convert to numpy array
        features = self.scaler.fit_transform(features)  # Scale features
        labels = LabelEncoder().fit_transform(labels['&-target'].values)
        
        # Convert to tensors once
        features_tensor = torch.tensor(features, dtype=torch.float32)
        labels_tensor = torch.tensor(labels, dtype=torch.long)
        
        # Create dataset
        dataset = CryptoDataset(features_tensor, labels_tensor)
        
        # Split dataset
        train_size = int(self.train_split * len(dataset))
        val_size = len(dataset) - train_size
        
        self.train_dataset, self.val_dataset = random_split(
            dataset, 
            [train_size, val_size],
            generator=torch.Generator().manual_seed(self.random_seed)
        )
        
        # Store dimensions
        self.input_dim = self.n_features
        self.output_dim = len(np.unique(labels))
        
        # Print setup info
        print(f"\nDataset splits:")
        print(f"Training set size: {train_size}")
        print(f"Validation set size: {val_size}")
        print(f"Input dimensions: {self.input_dim}")
        print(f"Output dimensions: {self.output_dim}")
        
    def train_dataloader(self) -> DataLoader:
        return DataLoader(
            self.train_dataset, 
            batch_size=self.batch_size,
            num_workers=self.num_workers, 
            shuffle=True,
            pin_memory=True,
            drop_last=True,
            persistent_workers=True if self.num_workers > 0 else False
        )

    def val_dataloader(self) -> DataLoader:
        return DataLoader(
            self.val_dataset, 
            batch_size=self.batch_size,
            num_workers=self.num_workers, 
            pin_memory=True,
            drop_last=True,
            persistent_workers=True if self.num_workers > 0 else False
        )

if __name__ == "__main__":
    # Initialize with deterministic behavior
    pl.seed_everything(42)
    
    # Initialize data module
    data_module = CryptoDataModule(
        directory_path='/allah/data/parquet',
        batch_size=64,
        num_workers=4,
        n_features=20
    )
    
    # Setup and test
    data_module.setup()
    
    # Test batch loading
    train_loader = data_module.train_dataloader()
    features, labels = next(iter(train_loader))
    
    print("\nBatch test results:")
    print(f"Features shape: {features.shape}")
    print(f"Labels shape: {labels.shape}")

Seed set to 42


Original features shape: (17721, 1207)

Randomly selected 20 features:
['%-plus_di-period_100_ETH/USDTUSDT_1m', '%-ad-period_100_shift-1_ETH/USDTUSDT_1m', '%-bb_upper-period_14_shift-3_ETH/USDTUSDT_15m', '%-obv-period_100_ETH/USDTUSDT_1m', '%-rsi-period_28_ETH/USDTUSDT_15m', '%-lower_shadow-period_100_ETH/USDTUSDT_15m', '%-bb_middle-period_14_shift-2_ETH/USDTUSDT_1m', '%-rsi-period_28_ETH/USDTUSDT_1m', '%-obv-period_14_shift-3_ETH/USDTUSDT_1m', '%-tema-period_14_shift-2_ETH/USDTUSDT_15m', '%-plus_di-period_14_shift-3_ETH/USDTUSDT_1m', '%-willr-period_100_shift-3_ETH/USDTUSDT_1m', '%-cci-period_14_shift-1_ETH/USDTUSDT_1m', '%-wma-period_100_shift-2_ETH/USDTUSDT_1m', '%-ohlc4-period_100_shift-2_ETH/USDTUSDT_15m', '%-stochf_k-period_28_ETH/USDTUSDT_1m', '%-ohlc4-period_28_shift-3_ETH/USDTUSDT_15m', '%-trix-period_28_shift-1_ETH/USDTUSDT_1m', '%-volume_change-period_100_shift-1_ETH/USDTUSDT_1m', '%-tema-period_28_shift-2_ETH/USDTUSDT_15m']

Dataset splits:
Training set size: 14176
Validati

In [10]:
class CryptoPricePredictor(pl.LightningModule):
    def __init__(self, input_dim=20, hidden_dims=[512, 256, 128], dropout_rate=0.3, learning_rate=1e-3):
        super().__init__()
        self.save_hyperparameters()
        
        # Initialize class weights
        self.register_buffer('class_weights', torch.tensor([1.0, 1.0]))
        
        # Model architecture
        layers = []
        dims = [input_dim] + hidden_dims
        
        for i in range(len(dims)-1):
            layers.extend([
                nn.Linear(dims[i], dims[i+1]),
                nn.BatchNorm1d(dims[i+1]),
                nn.LeakyReLU(0.1),
                nn.Dropout(dropout_rate)
            ])
        
        layers.append(nn.Linear(dims[-1], 2))
        self.model = nn.Sequential(*layers)
        self.learning_rate = learning_rate

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y, weight=self.class_weights)
        
        # Log training metrics
        self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.cross_entropy(logits, y, weight=self.class_weights)
        
        # Log validation loss
        self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate)
        return optimizer

In [12]:
if __name__ == "__main__":
    # Directory setup
    directory_path = '/allah/data/parquet'
    
    # Initialize data module
    data_module = CryptoDataModule(
        directory_path=directory_path,
        batch_size=64,
        num_workers=4,
        n_features=20
    )
    data_module.setup()

    # Model initialization
    model = CryptoPricePredictor(input_dim=data_module.input_dim)

    # Callbacks
    early_stopping = EarlyStopping(
        monitor='val_loss',
        mode='min',
        patience=15,
        min_delta=0.001,
        verbose=True
    )

    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        dirpath='checkpoints',
        filename='model-{epoch:02d}-{val_loss:.3f}',
        save_top_k=3,
        mode='min',
        verbose=True
    )

    # Initialize trainer
    trainer = Trainer(
        max_epochs=100,
        callbacks=[early_stopping, checkpoint_callback],
        logger=True,  # Use default logger
        accelerator='auto',
        devices=1,
        enable_progress_bar=True,
        enable_model_summary=True,
        log_every_n_steps=1
    )

    # Train model
    try:
        trainer.fit(model, datamodule=data_module)
        print(f"Best validation loss: {checkpoint_callback.best_model_score:.4f}")
    except Exception as e:
        print(f"Training error: {str(e)}")
        raise

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Original features shape: (17721, 1207)

Randomly selected 20 features:
['%-plus_di-period_100_ETH/USDTUSDT_1m', '%-ad-period_100_shift-1_ETH/USDTUSDT_1m', '%-bb_upper-period_14_shift-3_ETH/USDTUSDT_15m', '%-obv-period_100_ETH/USDTUSDT_1m', '%-rsi-period_28_ETH/USDTUSDT_15m', '%-lower_shadow-period_100_ETH/USDTUSDT_15m', '%-bb_middle-period_14_shift-2_ETH/USDTUSDT_1m', '%-rsi-period_28_ETH/USDTUSDT_1m', '%-obv-period_14_shift-3_ETH/USDTUSDT_1m', '%-tema-period_14_shift-2_ETH/USDTUSDT_15m', '%-plus_di-period_14_shift-3_ETH/USDTUSDT_1m', '%-willr-period_100_shift-3_ETH/USDTUSDT_1m', '%-cci-period_14_shift-1_ETH/USDTUSDT_1m', '%-wma-period_100_shift-2_ETH/USDTUSDT_1m', '%-ohlc4-period_100_shift-2_ETH/USDTUSDT_15m', '%-stochf_k-period_28_ETH/USDTUSDT_1m', '%-ohlc4-period_28_shift-3_ETH/USDTUSDT_15m', '%-trix-period_28_shift-1_ETH/USDTUSDT_1m', '%-volume_change-period_100_shift-1_ETH/USDTUSDT_1m', '%-tema-period_28_shift-2_ETH/USDTUSDT_15m']

Dataset splits:
Training set size: 14176
Validati

/allah/freqtrade/.venv/lib/python3.11/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /allah/data/parquet/checkpoints exists and is not empty.

  | Name  | Type       | Params | Mode 
---------------------------------------------
0 | model | Sequential | 177 K  | train
---------------------------------------------
177 K     Trainable params
0         Non-trainable params
177 K     Total params
0.708     Total estimated model params size (MB)
14        Modules in train mode
0         Modules in eval mode


Original features shape: (17721, 1207)

Randomly selected 20 features:
['%-plus_di-period_100_ETH/USDTUSDT_1m', '%-ad-period_100_shift-1_ETH/USDTUSDT_1m', '%-bb_upper-period_14_shift-3_ETH/USDTUSDT_15m', '%-obv-period_100_ETH/USDTUSDT_1m', '%-rsi-period_28_ETH/USDTUSDT_15m', '%-lower_shadow-period_100_ETH/USDTUSDT_15m', '%-bb_middle-period_14_shift-2_ETH/USDTUSDT_1m', '%-rsi-period_28_ETH/USDTUSDT_1m', '%-obv-period_14_shift-3_ETH/USDTUSDT_1m', '%-tema-period_14_shift-2_ETH/USDTUSDT_15m', '%-plus_di-period_14_shift-3_ETH/USDTUSDT_1m', '%-willr-period_100_shift-3_ETH/USDTUSDT_1m', '%-cci-period_14_shift-1_ETH/USDTUSDT_1m', '%-wma-period_100_shift-2_ETH/USDTUSDT_1m', '%-ohlc4-period_100_shift-2_ETH/USDTUSDT_15m', '%-stochf_k-period_28_ETH/USDTUSDT_1m', '%-ohlc4-period_28_shift-3_ETH/USDTUSDT_15m', '%-trix-period_28_shift-1_ETH/USDTUSDT_1m', '%-volume_change-period_100_shift-1_ETH/USDTUSDT_1m', '%-tema-period_28_shift-2_ETH/USDTUSDT_15m']

Dataset splits:
Training set size: 14176
Validati

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 14, global step 3315: 'val_loss' was not in top 3


Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 15 records. Best score: 0.652. Signaling Trainer to stop.
Epoch 15, global step 3536: 'val_loss' reached 0.65131 (best 0.65131), saving model to '/allah/data/parquet/checkpoints/model-epoch=15-val_loss=0.651.ckpt' as top 3


Best validation loss: 0.6513


In [ ]:
# Add new cell
import optuna
from typing import List, Dict, Any
import logging

class OptunaObjective:
    def __init__(self, data_module: CryptoDataModule):
        self.data_module = data_module
        
    def __call__(self, trial: optuna.Trial) -> float:
        # Define hyperparameters to optimize
        n_layers = trial.suggest_int('n_layers', 2, 4)
        hidden_dims = []
        for i in range(n_layers):
            hidden_dims.append(trial.suggest_int(f'hidden_dim_{i}', 64, 512, step=64))
        
        params = {
            'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
            'hidden_dims': hidden_dims,  # Store the full list
            'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5),
            'batch_size': trial.suggest_categorical('batch_size', [32, 64, 128, 256])
        }
        
        # Update data module batch size
        self.data_module.batch_size = params['batch_size']
        self.data_module.setup()
        
        # Initialize model with trial params
        model = CryptoPricePredictor(
            input_dim=self.data_module.input_dim,
            hidden_dims=params['hidden_dims'],
            dropout_rate=params['dropout_rate'],
            learning_rate=params['learning_rate']
        )
        
        # Training callbacks
        callbacks = [
            ModelCheckpoint(
                monitor="val_win_precision",
                mode="max",
                save_top_k=1,
                save_weights_only=True,
                verbose=False
            ),
            EarlyStopping(
                monitor="val_win_precision",
                mode="max",
                patience=5,
                verbose=False
            )
        ]
        
        # Initialize trainer
        trainer = Trainer(
            max_epochs=30,  # Reduced for optimization
            callbacks=callbacks,
            logger=False,  # Disable logging for optimization
            enable_progress_bar=False,
            accelerator='auto',
            devices=1
        )
        
        # Train and get best validation metric
        try:
            trainer.fit(model, datamodule=self.data_module)
            best_score = trainer.callback_metrics.get("val_win_precision", float('-inf'))
            if isinstance(best_score, torch.Tensor):
                best_score = best_score.item()
        except Exception as e:
            print(f"Trial failed: {e}")
            best_score = float('-inf')
            
        return best_score

def run_optuna_optimization(data_module: CryptoDataModule, n_trials: int = 50) -> Dict[str, Any]:
    # Create study
    study = optuna.create_study(
        direction="maximize",
        study_name="crypto_price_prediction",
        pruner=optuna.pruners.MedianPruner()
    )
    
    # Run optimization
    objective = OptunaObjective(data_module)
    study.optimize(objective, n_trials=n_trials)
    
    # Print optimization results
    print("\n=== Optimization Results ===")
    print(f"Best trial value: {study.best_trial.value:.4f}")
    print("\nBest hyperparameters:")
    for param, value in study.best_trial.params.items():
        print(f"{param}: {value}")
    
    return study.best_trial.params

if __name__ == "__main__":
    # Directory and data setup
    directory_path = '/allah/data/parquet'
    data_module = CryptoDataModule(directory_path=directory_path)
    data_module.setup()
    
    # Run hyperparameter optimization
    best_params = run_optuna_optimization(data_module, n_trials=50)
    
    # Train final model with best parameters
    final_model = CryptoPricePredictor(
        input_dim=data_module.input_dim,
        hidden_dims=best_params['hidden_dims'],
        dropout_rate=best_params['dropout_rate'],
        learning_rate=best_params['learning_rate']
    )
    
    # Final training callbacks
    final_callbacks = [
        ModelCheckpoint(
            monitor="val_win_precision",
            mode="max",
            filename="best-model-{epoch:02d}-{val_win_precision:.2f}",
            save_top_k=3,
            verbose=True
        ),
        EarlyStopping(
            monitor="val_win_precision",
            mode="max",
            patience=10,
            verbose=True
        ),
        LearningRateMonitor(logging_interval='epoch')
    ]
    
    # Final training
    final_trainer = Trainer(
        max_epochs=1000,
        callbacks=final_callbacks,
        logger=TensorBoardLogger(save_dir=directory_path, name="final_model_logs"),
        accelerator='auto',
        devices=1
    )
    
    # Train final model
    final_trainer.fit(final_model, datamodule=data_module)
    print(f"\nFinal model best validation precision: {final_trainer.callback_metrics['val_win_precision']:.4f}")

In [ ]:
# Add new cell
import ray
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
from ray.tune.integration.pytorch_lightning import TuneReportCallback
import os
from functools import partial

class TuneReportCallbackMetrics(TuneReportCallback):
    def on_validation_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        metrics = {k: v.item() if hasattr(v, 'item') else v for k, v in metrics.items()}
        self._report(metrics)

def train_tune_model(config, data_module=None, num_epochs=30):
    model = CryptoPricePredictor(
        input_dim=data_module.input_dim,
        hidden_dims=config["hidden_dims"],
        dropout_rate=config["dropout_rate"],
        learning_rate=config["learning_rate"]
    )
    
    data_module.batch_size = config["batch_size"]
    data_module.setup()
    
    callbacks = [
        TuneReportCallbackMetrics(
            metrics={
                "val_win_precision": "val_win_precision",
                "val_loss": "val_loss"
            },
            on="validation_end"
        ),
        EarlyStopping(
            monitor="val_loss",
            mode="min",
            patience=3,
            verbose=False
        )
    ]
    
    trainer = Trainer(
        max_epochs=num_epochs,
        callbacks=callbacks,
        logger=False,
        enable_progress_bar=False,
        accelerator='cpu',
        devices=1,
        gradient_clip_val=1.0,
        accumulate_grad_batches=2
    )
    
    trainer.fit(model, datamodule=data_module)

def run_ray_optimization(data_module, num_samples=20, num_epochs=20):
    # Initialize Ray with updated configuration
    ray.init(
        ignore_reinit_error=True,
        runtime_env={
            "env_vars": {
                "RAY_memory_monitor_refresh_ms": "0",
                "RAY_memory_usage_threshold": "0.95"
            }
        },
        _memory=2000 * 1024 * 1024
    )
    
    config = {
        "learning_rate": tune.loguniform(1e-4, 1e-2),
        "hidden_dims": tune.sample_from(lambda _: [
            tune.randint(64, 256).sample() for _ in range(2)
        ]),
        "dropout_rate": tune.uniform(0.1, 0.5),
        "batch_size": tune.choice([16, 32, 64])
    }
    
    scheduler = ASHAScheduler(
        max_t=num_epochs,
        grace_period=3,
        reduction_factor=3,
        brackets=1
    )
    
    reporter = CLIReporter(
        parameter_columns=["learning_rate", "dropout_rate", "batch_size"],
        metric_columns=["val_win_precision", "val_loss", "training_iteration"]
    )
    
    train_fn = partial(train_tune_model, data_module=data_module, num_epochs=num_epochs)
    
    # Updated tune.run with storage_path instead of local_dir
    analysis = tune.run(
        train_fn,
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter,
        name="ray_tune_crypto",
        metric="val_win_precision",
        mode="max",
        resources_per_trial={
            "cpu": 1,
            "gpu": 0
        },
        storage_path="/tmp/ray_results",  # Changed from local_dir to storage_path
        max_concurrent_trials=2,
        raise_on_failed_trial=False
    )
    
    best_trial = analysis.best_trial
    best_config = best_trial.config
    
    print("\n=== Ray Tune Optimization Results ===")
    print(f"Best trial config: {best_config}")
    print(f"Best trial final validation precision: {best_trial.last_result['val_win_precision']:.4f}")
    
    ray.shutdown()
    return best_config

if __name__ == "__main__":
    directory_path = '/allah/data/parquet'
    data_module = CryptoDataModule(directory_path=directory_path)
    data_module.setup()
    
    try:
        best_params = run_ray_optimization(data_module, num_samples=20, num_epochs=20)
        
        final_model = CryptoPricePredictor(
            input_dim=data_module.input_dim,
            hidden_dims=best_params['hidden_dims'],
            dropout_rate=best_params['dropout_rate'],
            learning_rate=best_params['learning_rate']
        )
        
        final_callbacks = [
            ModelCheckpoint(
                monitor="val_win_precision",
                mode="max",
                filename="best-model-{epoch:02d}-{val_win_precision:.2f}",
                save_top_k=1,
                save_weights_only=True
            ),
            EarlyStopping(
                monitor="val_win_precision",
                mode="max",
                patience=5,
                verbose=True
            )
        ]
        
        final_trainer = Trainer(
            max_epochs=100,
            callbacks=final_callbacks,
            logger=TensorBoardLogger(save_dir=directory_path, name="final_model_logs"),
            accelerator='cpu',
            devices=1,
            gradient_clip_val=1.0
        )
        
        final_trainer.fit(final_model, datamodule=data_module)
        print(f"\nFinal model best validation precision: {final_trainer.callback_metrics['val_win_precision']:.4f}")
        
    except Exception as e:
        print(f"Error during optimization: {e}")
    finally:
        ray.shutdown()